In [1]:
%load_ext autoreload 
%autoreload 2

import operator
import sys

import matplotlib.pyplot as plt
import numpy as np

sys.path.insert(0, '../') # The following modules are in the directory above
from cohort_model import (
    run_cohort_simulation,
    HYP_WILDTYPE, 
    MUT_CAPTIVITY,
)
from figures import ROOT_DIR

# Parameter estimation for case study 1 (*Frontinella pyramitela*)

We have estimated model parameters using least squares fitting. Typically, such problems are solved with utilities such as [`scipy.optimize.curve_fit`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html). However, our model function is non-analytical, due to its stochasticity. Thus, analytical curve fitting methods are not possible to use, and we have instead opted for a simple iterative search in the parameter space.

We encourage examining other methods of exploring the parameter space. This could potentially be a conventional space search algorithm, or an evolutionary algorithm framework such as [DEAP](https://deap.readthedocs.io/en/master/). If you develop a more efficient/optimal method, we welcome you to submit a pull request on GitHub.

**Note:** The parameter values used in figures 1 and 2 were obtained during an earlier least squares fitting. In restructuring the code for publication we have also improved the fitting procedure and obtained new and better values. However, these are not used in the figures in order to produce results matching those in the published paper (which could not be updated by the time we obtained the better fit).

## Estimating $\alpha$ and $\kappa$

In [2]:
# Simulation parameters
individual_count = 1000 
repetition_count = 100  

t_m_captivity = 202  # Max t for curve fitting (last x data point = 201) 
t_m_wt = 101         # Max t for curve fitting (last x data point = 100)     

In [3]:
captivity_x = np.genfromtxt(f'{ROOT_DIR}/data/austad_1989/captivity_x.txt')
captivity_y = np.genfromtxt(f'{ROOT_DIR}/data/austad_1989/captivity_y.txt')

xdata = np.round(captivity_x).astype('int64') # In order to use for indexing
ydata = captivity_y * individual_count

In [5]:
%%time
data_count = len(xdata)

fit = []
# TODO: The population is set to mutant captivity in order to 
for alpha in np.arange(0.0002, 0.0003, 0.00001): 
    for kappa in np.arange(0.02, 0.04, 0.0001): 
        hazard_rate_params = dict(alpha=alpha, kappa=kappa, population=MUT_CAPTIVITY)
        population_survivorship = run_cohort_simulation(
            repetition_count, 
            individual_count, 
            hazard_rate_params, 
            t_m_captivity
        )
        mean = np.mean(population_survivorship, axis=0)[xdata]
        squares = [(mean[index] - ydata[index])**2 for index in range(data_count)] 
        fit.append((alpha, kappa, sum(squares)))

CPU times: user 2h 43min 12s, sys: 2h 45min 35s, total: 5h 28min 48s
Wall time: 5h 31min 1s


In [6]:
best_fits = sorted(fit, key=operator.itemgetter(2))
print(*best_fits[0:10], sep='\n')

(0.0002, 0.03349999999999992, 1093.8604000000005)
(0.00021, 0.03309999999999992, 1130.759000000001)
(0.0002, 0.03359999999999992, 1143.5305000000028)
(0.00021, 0.033199999999999924, 1175.9992999999972)
(0.0002, 0.03369999999999992, 1192.7424999999978)
(0.00022, 0.03289999999999992, 1194.2436)
(0.0002, 0.03379999999999991, 1248.6812999999984)
(0.00021, 0.033399999999999916, 1268.3848999999973)
(0.00022, 0.032699999999999924, 1297.2533999999987)
(0.00021, 0.03329999999999992, 1315.9933999999994)


## Estimating $\epsilon$ and $h_{wt}(t)$

In [7]:
x_wild = np.genfromtxt(f'{ROOT_DIR}/data/austad_1989/wild_x.txt')
y_wild = np.genfromtxt(f'{ROOT_DIR}/data/austad_1989/wild_y.txt')

xdata_w = np.round(x_wild).astype('int64') # In order to use for indexing
ydata_w = y_wild * individual_count

xdata_w = xdata_w[:-2] # In order not to fit to the last two data points
ydata_w = ydata_w[:-2] # In order not to fit to the last two data points

In [13]:
%%time
n = len(xdata_w)

fit = []
# TODO: The population is set to the hypothetical wild type, in order to use the 
for prod_wt in np.arange(0.0378, 0.0382, 0.000025):  # prod_wt = (1 - epsilon) * h_wt
    hazard_rate_params = dict(hazard_rate_wt=prod_wt, population=HYP_WILDTYPE)
    population_survivorship = run_cohort_simulation(
        repetition_count, 
        individual_count, 
        hazard_rate_params, 
        t_m_wt,
    )
    mean = np.mean(population_survivorship, axis=0)[xdata_w]
    squares = [(mean[i] - ydata_w[i])**2 for i in range(n)] # Not fitting to last two data points
    fit.append((prod_wt, sum(squares)))

CPU times: user 22.9 s, sys: 18.5 s, total: 41.3 s
Wall time: 41.6 s


In [14]:
best_fits = sorted(fit, key=operator.itemgetter(1))
print(*best_fits[0:10], sep='\n')

(0.03797499999999998, 1045.1383999999973)
(0.03789999999999999, 1054.7347999999981)
(0.037924999999999986, 1066.2092999999975)
(0.0378, 1075.0033999999994)
(0.03799999999999998, 1090.0588999999966)
(0.03787499999999999, 1103.3432999999966)
(0.038024999999999975, 1105.7740999999994)
(0.037849999999999995, 1115.4339999999977)
(0.037825, 1126.9093999999943)
(0.037949999999999984, 1133.3473999999987)


In [15]:
prod_wt = best_fits[0][0]
for epsilon in np.arange(0.01, 0.05, 0.01):
    h_wt = prod_wt / (1 - epsilon)
    print(f'epsilon = {epsilon}, h_wt = {h_wt}')

epsilon = 0.01, h_wt = 0.03835858585858584
epsilon = 0.02, h_wt = 0.03874999999999998
epsilon = 0.03, h_wt = 0.039149484536082454
epsilon = 0.04, h_wt = 0.03955729166666665
